In [46]:
import pandas as pd

In [47]:
auna_clean = pd.read_csv('auna_clean.csv')
internacional_clean = pd.read_csv('internacional_clean.csv')
doctors_clean = pd.read_csv('DOCTORES_CLEAN.csv')

In [48]:
# doctors_clean.head()
auna_clean.head()

,CMP,name,photo,loc_id_arrays
0,60449,Acevedo Tizón Anaïs Caterina,https://web-auna-backend-prd-images.s3.amazona...,['CLIN-3']
1,45225,Acosta Chacaltana Max Carlos,https://web-auna-backend-prd-images.s3.amazona...,['CLIN-3']
2,48934,Acuña Barrueto Frank Leopoldo,https://web-auna-backend-prd-images.s3.amazona...,['CLIN-3']
3,27289,Aguilar Segura Jorge Arturo,https://web-auna-backend-prd-images.s3.amazona...,['CLIN-3']
4,39031,Ajalcriña Guerrero Hernán Roberto,https://web-auna-backend-prd-images.s3.amazona...,"['CLIN-55', 'CLIN-3']"


In [49]:
doctors_auna = doctors_clean.merge(
    auna_clean[['CMP', 'loc_id_arrays']],
    how='left',
    left_on='CMP',
    right_on='CMP'
)

# doctors_auna.rename(columns={'loc_id_arrays': 'clinic_ids'})
# doctors_auna = doctors_auna.dropna(subset=['clinic_ids'])
# doctors_auna.to_csv('doctors_auna_clean.csv', index=False)
doctors_auna

,CMP,nombres,apellido_paterno,apellido_materno,status,foto_url,especialidad_id,loc_id_arrays
0,21,Jorge Enrique,Berrios,Reiterer,HÁBIL,fotos/00021.jpg,29,NaN
1,22,Fernando,Robinson,Gunther,HÁBIL,fotos/00022.jpg,46,NaN
2,33,Ricardo Jorge,Abugattas,Jasaui,HÁBIL,fotos/00033.jpg,8,NaN
3,57,Juan Alfredo,Jara,Velarde,HÁBIL,fotos/00057.jpg,36,NaN
4,69,Elvira Rosa,Giro,Palau,HÁBIL,fotos/00069.jpg,121,NaN
...,...,...,...,...,...,...,...,...
33810,110561,Alexis,Castillo,Belen,HÁBIL,fotos/110561.jpg,29,NaN
33811,111259,Hugo,Medina,Jura,HÁBIL,fotos/111259.jpg,8,NaN
33812,111432,Maria Alessandra,Cano,Blaires,HÁBIL,fotos/111432.jpg,38,NaN
33813,113717,Juan Guillermo,Estrada,Londoño,HÁBIL,fotos/113717.jpg,44,NaN


In [50]:
internacional_clean.head()
# First, strip the rightmost '0's, then convert to int
# Only keep rows where 'CMP' is numeric, drop others (e.g., values like '01-014')
internacional_clean = internacional_clean.rename(columns={'cmp': 'CMP'})
internacional_clean = internacional_clean[pd.to_numeric(internacional_clean['CMP'], errors='coerce').notnull()]
internacional_clean['CMP'] = internacional_clean['CMP'].astype(int)




In [51]:
internacional_clean

,CMP,url_image,loc_id_arrays
0,34572,https://sabrmeamundociprod.blob.core.windows.n...,"['CLIN-4', 'CLIN-5']"
1,16284,https://sabrmeamundociprod.blob.core.windows.n...,['CLIN-52']
2,15719,https://sabrmeamundociprod.blob.core.windows.n...,"['CLIN-5', 'CLIN-53']"
3,13978,https://sabrmeamundociprod.blob.core.windows.n...,['CLIN-5']
4,17823,https://sabrmeamundociprod.blob.core.windows.n...,['CLIN-4']
...,...,...,...
978,75091,https://sabrmeamundociprod.blob.core.windows.n...,"['CLIN-4', 'CLIN-5']"
979,53999,https://sabrmeamundociprod.blob.core.windows.n...,['CLIN-4']
980,56364,https://sabrmeamundociprod.blob.core.windows.n...,"['CLIN-52', 'CLIN-53']"
981,56880,https://sabrmeamundociprod.blob.core.windows.n...,"['CLIN-5', 'CLIN-52']"


In [63]:
doctors_internacional = doctors_auna.merge(
    internacional_clean[['CMP', 'loc_id_arrays']],
    how='left',
    left_on='CMP',
    right_on='CMP'
)

In [64]:
import ast

def merge_arrays(row):
    x = row['loc_id_arrays_x']
    y = row['loc_id_arrays_y']
    # Parse string representation of lists if necessary
    x_list = ast.literal_eval(x) if isinstance(x, str) and x.startswith('[') else (x if isinstance(x, list) else [] if pd.isna(x) else [x])
    y_list = ast.literal_eval(y) if isinstance(y, str) and y.startswith('[') else (y if isinstance(y, list) else [] if pd.isna(y) else [y])
    if x_list and y_list:
        return str(list(x_list) + list(y_list))
    elif x_list:
        return str(list(x_list))
    elif y_list:
        return str(list(y_list))
    else:
        return None

doctors_internacional['loc_id_arrays'] = doctors_internacional.apply(merge_arrays, axis=1)



In [65]:
doctors_internacional.drop(columns=['loc_id_arrays_x', 'loc_id_arrays_y'], inplace=True)
doctors_internacional = doctors_internacional[doctors_internacional['loc_id_arrays'].notna()]


In [66]:
doctors_internacional.to_csv('DOCTORES_COMPLETOS.csv', index=False)